<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 2 Assignment 2*

# Document Representations: Bag-Of-Words

In [1]:
import re
import string

!pip install -U nltk

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize # Sentence Tokenizer
from nltk.tokenize import word_tokenize # Word Tokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.probability import FreqDist

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

Requirement already up-to-date: nltk in /home/seek/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages (3.4.4)


[nltk_data] Downloading package punkt to /home/seek/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/seek/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# 1) (optional) Scrape 100 Job Listings that contain the title "Data Scientist" from indeed.com

At a minimum your final dataframe of job listings should contain
- Job Title
- Job Description

If you choose to not to scrape the data, there is a CSV with outdated data in the directory. Remeber, if you scrape Indeed, you're helping yourself find a job. ;)

In [2]:
##### Your Code Here #####

# Pulling job search query from indeed.com
import requests
import bs4
from bs4 import BeautifulSoup
import time

URL = 'https://www.indeed.com/jobs?q=data+scientist+%2420%2C000&start=10'
#conducting a request of the stated URL above:
page = requests.get(URL)
#specifying a desired format of “page” using the html parser - this allows python to read the various components of the page, rather than treating it as one long string.
soup = BeautifulSoup(page.text, 'html.parser')
#printing soup in a more structured tree format that makes for easier reading
print(soup.prettify())


<!DOCTYPE html>
<html dir="ltr" lang="en">
 <head>
  <meta content="text/html;charset=utf-8" http-equiv="content-type"/>
  <script src="/s/e7e2a1c/en_US.js" type="text/javascript">
  </script>
  <script>
   window.indeed=window.indeed||{};indeed.crashtext=indeed.crashtext||{};indeed.crashtext.populate=indeed.crashtext.populate||function(c,g){var d=window[g];if(d){var e=d[""]||{},h=e.salt;if(e.hasOwnProperty("salt"))for(var b in c){for(var a=void 0,a=h,f=b.length;f;)a=33*a^b.charCodeAt(--f);a>>>=0;e.hasOwnProperty("id_length")&&(a=String(a).substring(0,e.id_length));d[a]=c[b]}else for(b in c)d[b]=[null].concat(c[b])}};indeed.crashtext.populate({"indeedapply_serp_label":["Easy apply"],"Get Job Alerts for this search":["See new jobs for this search"],"Create job alert":["Turn on"]}, 'indeed.i18n.localeData')
  </script>
  <link href="/s/684a333/jobsearch_all.css" rel="stylesheet" type="text/css"/>
  <link href="http://rss.indeed.com/rss?q=data+scientist+%2420%2C000" rel="alternate" title=

In [7]:
# Refinning search query by job title and locations

import requests
import bs4
from bs4 import BeautifulSoup
import time

jobs = []
companies = []
locations = []
summaries = []

max_results_per_city = 50


city_set = ['New+York','Chicago','San+Francisco', 'Austin', 'Seattle', 'Los+Angeles', 
            'Philadelphia', 'Atlanta', 'Dallas', 'Pittsburgh', 'Portland', 'Phoenix', 
            'Denver', 'Houston', 'Miami', 'Washington+DC', 'Boulder', 'Greenville', 
            'Columbia', 'Charleston', 'Charlotte', 'Savannah','Spartanburg', 'Anderson',
           'Fort+Wayne', 'Indianapolis']

for city in city_set:
      for start in range(0, max_results_per_city, 10):
        page = requests.get("https://www.indeed.com/jobs?q=data+scientist&l="
                            + str(city) + "&start=" + str(start))
        time.sleep(1)                      
        soup = BeautifulSoup(page.text, "lxml") 
        # for each row
        for div in soup.find_all(name="div", attrs={"class":"row"}):
            # Job title
            for i in div.findAll(name="a", attrs={"data-tn-element":"jobTitle"}):
                job_title = i["title"]
                if len(job_title) > 0:
                    jobs.append(job_title)
                else:
                    jobs.append('not found')
            # Company    
            for i in div.findAll(name="span", attrs={"class":"company"}):
                company = i.text
                if len(company) > 0:
                    companies.append(company)
                else:
                    companies.append('not found')
            # Location    
            for i in div.findAll("div", attrs={"class": "recJobLoc"}):
                location = i['data-rc-loc']
                if len(location) > 0:
                    locations.append(location)
                else:
                    location.append('not found')
            # Summary  
            for i in div.findAll("div", attrs={"class": "summary"}):
                summary = i.text
                if len(summary) > 0:
                    summaries.append(summary)
                else:
                    summary.append('not found')

print(len(jobs), len(companies), len(locations), len(summaries))

1924 1924 1924 1924


In [8]:
jobs = pd.DataFrame({'job_title':jobs,
          'company':companies,
          'location':locations,
          'summary':summaries})

jobs.head()

job_title                                         company  \
0      Data Scientist           \n\n        Disney Streaming Services   
1    Data Scientist I                                 \n\n        AIG   
2      Data Scientist  \n\n        VISITING NURSE SERVICE OF NEW YORK   
3  Sr. Data Scientist              \n\n        Ascensia Diabetes Care   
4       ARD SCIENTIST                       \n        Epic Pharma LLC   

                  location                                            summary  
0             New York, NY  \n            The Data Scientist is a critical...  
1             New York, NY  \nData Scientists collaborate with AIG’s Life ...  
2            Manhattan, NY  \n            What partnerships can we develop...  
3             Valhalla, NY  \n            Experience in and understanding ...  
4  Springfield Gardens, NY  \nData review. Health insurance. Analytical Re...

In [32]:
jobs['company'] = jobs['company'].str.replace('[^a-zA-Z ^0-9]', '')

jobs['summary'] = jobs['summary'].str.replace('[^a-zA-Z ^0-9]', '')

jobs.head()

job_title                                     company  \
0      Data Scientist                   Disney Streaming Services   
1    Data Scientist I                                         AIG   
2      Data Scientist          VISITING NURSE SERVICE OF NEW YORK   
3  Sr. Data Scientist                      Ascensia Diabetes Care   
4       ARD SCIENTIST                             Epic Pharma LLC   

                  location                                            summary  \
0             New York, NY              The Data Scientist is a critical p...   
1             New York, NY  Data Scientists collaborate with AIGs Life and...   
2            Manhattan, NY              What partnerships can we develop i...   
3             Valhalla, NY              Experience in and understanding of...   
4  Springfield Gardens, NY  Data review Health insurance Analytical Resear...   

                                   tokenized summary  
0  (            , The, Data, Scientist, is, a, cr...  
1  (Data, Scientists, collaborate, with, AIGs, Li...  
2  (            , What, partnerships, can, we, de...  
3  (            , Experience, in, and, understand...  
4  (Data, review, Health, insurance, Analytical, ...

In [10]:
jobs.to_csv('jobs.csv', index=False)

In [49]:
jobs.shape

(1924, 5)

## 2) Use Spacy to tokenize / clean the listings 

In [11]:
##### Your Code Here #####
import spacy
from spacy.tokenizer import Tokenizer

nlp = spacy.load("en_core_web_lg", parser=False, entity=False)

# Tokenizer
tokenizer = Tokenizer(nlp.vocab)

In [51]:
from spacy.tokenizer import Tokenizer
tokenizer = Tokenizer(nlp.vocab)

tokens = []

""" Make them tokens """
for doc in tokenizer.pipe(jobs['summary'], batch_size=500):
    doc_tokens = [token.text for token in doc]
    tokens.append(doc_tokens)

In [53]:
jobs['tokenized summary'] = tokens

In [ ]:
jobs['tokenized summary'].head()

In [54]:
jobs.head()

job_title                                     company  \
0      Data Scientist                   Disney Streaming Services   
1    Data Scientist I                                         AIG   
2      Data Scientist          VISITING NURSE SERVICE OF NEW YORK   
3  Sr. Data Scientist                      Ascensia Diabetes Care   
4       ARD SCIENTIST                             Epic Pharma LLC   

                  location                                            summary  \
0             New York, NY              The Data Scientist is a critical p...   
1             New York, NY  Data Scientists collaborate with AIGs Life and...   
2            Manhattan, NY              What partnerships can we develop i...   
3             Valhalla, NY              Experience in and understanding of...   
4  Springfield Gardens, NY  Data review Health insurance Analytical Resear...   

                                   tokenized summary  
0  [            , The, Data, Scientist, is, a, cr...  
1  [Data, Scientists, collaborate, with, AIGs, Li...  
2  [            , What, partnerships, can, we, de...  
3  [            , Experience, in, and, understand...  
4  [Data, review, Health, insurance, Analytical, ...

# 3) Use Scikit-Learn's CountVectorizer to get word counts for each listing.

In [ ]:
##### Your Code Here #####
raise Exception("\nThis task is not complete. \nReplace this line with your code for the task."

# 4) Visualize the most common word counts

In [ ]:
##### Your Code Here #####
raise Exception("\nThis task is not complete. \nReplace this line with your code for the task."

 # 5) Use Scikit-Learn's tfidfVectorizer to get a TF-IDF feature matrix

In [ ]:
##### Your Code Here #####
raise Exception("\nThis task is not complete. \nReplace this line with your code for the task."

 # 6) Create a NearestNeighbor Model. Write the description of your ideal datascience job and query your job listings. 

In [ ]:
##### Your Code Here #####
raise Exception("\nThis task is not complete. \nReplace this line with your code for the task."

## Stretch Goals

 - Try different visualizations for words and frequencies - what story do you want to tell with the data?
 - Scrape Job Listings for the job title "Data Analyst". How do these differ from Data Scientist Job Listings
 - Try and identify requirements for experience specific technologies that are asked for in the job listings. How are those distributed among the job listings?
 - Use a clustering algorithm to cluster documents by their most important terms. Do the clusters reveal any common themes?
  - **Hint:** K-means might not be the best algorithm for this. Do a little bit of research to see what might be good for this. Also, remember that algorithms that depend on Euclidean distance break down with high dimensional data.